In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [3]:
df_ind_fluxo_educ = spark.read.format('parquet').load('s3a://silver/educacao_superior/indicadores_fluxo_educacao_superior')
print('==========> df_ind_fluxo_educ.printSchema(): ', df_ind_fluxo_educ.printSchema())
print('==========> df_ind_fluxo_educ.count(): ', df_ind_fluxo_educ.count())
print('==========> df_ind_fluxo_educ.show(5, False): ', df_ind_fluxo_educ.show(5, False))

root
 |-- codigo_instituicao: long (nullable = true)
 |-- nome_instituicao: string (nullable = true)
 |-- categoria_administrativa: integer (nullable = true)
 |-- organizacao_academica: integer (nullable = true)
 |-- codigo_curso_graduacao: long (nullable = true)
 |-- nome_curso_graduacao: string (nullable = true)
 |-- codigo_regiao_geografica_curso: integer (nullable = true)
 |-- nome_regiao_geografica_curso: string (nullable = true)
 |-- codigo_unidade_federativa_curso: integer (nullable = true)
 |-- nome_unidade_federativa_curso: string (nullable = true)
 |-- codigo_municipio_curso: long (nullable = true)
 |-- nome_municipio_curso: string (nullable = true)
 |-- tipo_grau_academico: integer (nullable = true)
 |-- tipo_modalidade_ensino: integer (nullable = true)
 |-- codigo_area_curso_classificacao_cine_brasil: string (nullable = true)
 |-- nome_area_curso_classificacao_cine_brasil: string (nullable = true)
 |-- codigo_grande_area_curso_classificacao_cine_brasil: integer (nullable = 

In [8]:
df_ind_fluxo_educ_gold = df_ind_fluxo_educ.select(
        fn.col('codigo_instituicao'),
        fn.col('nome_instituicao'),
        fn.col('categoria_administrativa').alias('codigo_categoria_administrativa'),
        fn.when(fn.col('categoria_administrativa') == "1","Pública Federal")
          .when(fn.col('categoria_administrativa') == "2","Pública Estadual")
          .when(fn.col('categoria_administrativa') == "3","Pública Municipal")
          .when(fn.col('categoria_administrativa') == "4","Privada com Fins Lucrativos")
          .when(fn.col('categoria_administrativa') == "5","Privada sem Fins Lucrativos")
          .when(fn.col('categoria_administrativa') == "7","Especial")
          .otherwise('Outras')
          .alias('nome_categoria_administrativa'),
        fn.col('organizacao_academica').alias('codigo_organizacao_academica'),
        fn.when(fn.col('organizacao_academica') == "1","Universidade")
          .when(fn.col('organizacao_academica') == "2","Centro Universitário")
          .when(fn.col('organizacao_academica') == "3","Faculdade")
          .when(fn.col('organizacao_academica') == "4","Instituto Federal de Educação, Ciência e Tecnologia")
          .when(fn.col('organizacao_academica') == "5","Centro Federal de Educação Tecnológica")
          .otherwise('Outras')
          .alias('nome_organizacao_academica'),
        fn.col('codigo_curso_graduacao'),
        fn.col('nome_curso_graduacao'),
        fn.col('codigo_regiao_geografica_curso'),
        fn.col('nome_regiao_geografica_curso'),
        fn.col('codigo_unidade_federativa_curso'),
        fn.col('nome_unidade_federativa_curso'),
        fn.col('codigo_municipio_curso'),
        fn.col('nome_municipio_curso'),
        fn.col('tipo_grau_academico').alias('codigo_tipo_grau_academico'),
        fn.when(fn.col('tipo_grau_academico') == "1","Bacharelado")
          .when(fn.col('tipo_grau_academico') == "2","Licenciatura")
          .when(fn.col('tipo_grau_academico') == "3","Tecnológico")
          .otherwise('Outros')
          .alias('nome_tipo_grau_academico'),
        fn.col('tipo_modalidade_ensino').alias('codigo_tipo_modalidade_ensino'),
        fn.when(fn.col('tipo_modalidade_ensino') == "1","Presencial")
          .when(fn.col('tipo_modalidade_ensino') == "2","Curso a Distância")
          .otherwise('Outros')
          .alias('nome_tipo_modalidade_ensino'),
        fn.col('codigo_area_curso_classificacao_cine_brasil'),
        fn.col('nome_area_curso_classificacao_cine_brasil'),
        fn.col('codigo_grande_area_curso_classificacao_cine_brasil'),
        fn.col('nome_grande_area_curso_classificacao_cine_brasil'),
        fn.col('ano_ingresso'),
        fn.col('ano_referencia'),
        fn.col('prazo_integralizacao_anos'),
        fn.col('ano_integralizacao_curso'),
        fn.col('prazo_acompanhamento_curso_anos'),
        fn.col('ano_maximo_acompanhamento_curso'),
        fn.col('quantidade_ingressante_curso'),
        fn.col('quantidade_permanencia_curso_ano_referencia'),
        fn.col('quantidade_concluinte_curso_ano_referencia'),
        fn.col('quantidade_desistencia_curso_ano_referencia'),
        fn.col('quantidade_falecimento_curso_ano_referencia'),
        fn.col('taxa_permanencia_tap'),
        fn.col('taxa_conclusao_acumulada_tca'),
        fn.col('taxa_desistencia_acumulada_tda'),
        fn.col('taxa_conclusso_anual_tcan'),
        fn.col('taxa_desistencia_anual_tada')
)

In [ ]:
# (df_ind_fluxo_educ_gold
#  .write
#  .format('parquet')
#  .mode('overwrite')
#  .save('s3a://gold/educacao_superior/indicadores_fluxo_educacao_superior')
#  )

In [4]:
df_ind_fluxo_educ_gold = spark.read.format('parquet') \
   .load('s3a://gold/educacao_superior/indicadores_fluxo_educacao_superior')

In [6]:
print('==========> df_ind_fluxo_educ_gold.show(5, False): ',
      df_ind_fluxo_educ_gold.show(5, False))

+------------------+-----------------------------------+-------------------------------+-----------------------------+----------------------------+--------------------------+----------------------+--------------------+------------------------------+----------------------------+-------------------------------+-----------------------------+----------------------+--------------------+--------------------------+------------------------+-----------------------------+---------------------------+-------------------------------------------+-----------------------------------------+--------------------------------------------------+------------------------------------------------+------------+--------------+-------------------------+------------------------+-------------------------------+-------------------------------+----------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------+-----------------

<b>Categoria Adminsitrativa</b>

In [9]:
(df_ind_fluxo_educ_gold.groupBy(
    "codigo_categoria_administrativa", 
    "nome_categoria_administrativa").agg(fn.count('*').alias("qtde")) 
    .orderBy("codigo_categoria_administrativa")).show(15, False)

+-------------------------------+-----------------------------+-----+
|codigo_categoria_administrativa|nome_categoria_administrativa|qtde |
+-------------------------------+-----------------------------+-----+
|1                              |Pública Federal              |47300|
|2                              |Pública Estadual             |29440|
|3                              |Pública Municipal            |3840 |
|4                              |Privada com Fins Lucrativos  |90080|
|5                              |Privada sem Fins Lucrativos  |87270|
|7                              |Especial                     |1300 |
+-------------------------------+-----------------------------+-----+



<b>Organização Acadêmica</b>

In [10]:
(df_ind_fluxo_educ_gold.groupBy(
    "codigo_organizacao_academica", 
    "nome_organizacao_academica").agg(fn.count('*').alias("qtde")) 
    .orderBy("codigo_organizacao_academica")).show(15, False)

+----------------------------+---------------------------------------------------+------+
|codigo_organizacao_academica|nome_organizacao_academica                         |qtde  |
+----------------------------+---------------------------------------------------+------+
|1                           |Universidade                                       |131300|
|2                           |Centro Universitário                               |50860 |
|3                           |Faculdade                                          |70260 |
|4                           |Instituto Federal de Educação, Ciência e Tecnologia|6500  |
|5                           |Centro Federal de Educação Tecnológica             |310   |
+----------------------------+---------------------------------------------------+------+



<b>Grau Acadêmico</b>

In [11]:
(df_ind_fluxo_educ_gold.groupBy(
    "codigo_tipo_grau_academico", 
    "nome_tipo_grau_academico").agg(fn.count('*').alias("qtde")) 
    .orderBy("codigo_tipo_grau_academico")).show(15, False)

+--------------------------+------------------------+------+
|codigo_tipo_grau_academico|nome_tipo_grau_academico|qtde  |
+--------------------------+------------------------+------+
|1                         |Bacharelado             |151470|
|2                         |Licenciatura            |64080 |
|3                         |Tecnológico             |43680 |
+--------------------------+------------------------+------+



<b>Modalidade de Ensino</b>

In [12]:
(df_ind_fluxo_educ_gold.groupBy(
    "codigo_tipo_modalidade_ensino", 
    "nome_tipo_modalidade_ensino").agg(fn.count('*').alias("qtde")) 
    .orderBy("codigo_tipo_modalidade_ensino")).show(15, False)

+-----------------------------+---------------------------+------+
|codigo_tipo_modalidade_ensino|nome_tipo_modalidade_ensino|qtde  |
+-----------------------------+---------------------------+------+
|1                            |Presencial                 |252260|
|2                            |Curso a Distância          |6970  |
+-----------------------------+---------------------------+------+



<b>Região Geográfica</b>

In [13]:
(df_ind_fluxo_educ_gold.groupBy(
    "codigo_regiao_geografica_curso", 
    "nome_regiao_geografica_curso").agg(fn.count('*').alias("qtde")) 
    .orderBy("codigo_regiao_geografica_curso")).show(15, False)

+------------------------------+----------------------------+------+
|codigo_regiao_geografica_curso|nome_regiao_geografica_curso|qtde  |
+------------------------------+----------------------------+------+
|null                          |null                        |6970  |
|1                             |Norte                       |16790 |
|2                             |Nordeste                    |45640 |
|3                             |Sudeste                     |118940|
|4                             |Sul                         |47870 |
|5                             |Centro-Oeste                |23020 |
+------------------------------+----------------------------+------+



<b>Anos de Referência</b>

In [14]:
(df_ind_fluxo_educ_gold.groupBy(
    "ano_referencia").agg(fn.count('*').alias("qtde")) 
    .orderBy("ano_referencia")).show(15, False)

+--------------+-----+
|ano_referencia|qtde |
+--------------+-----+
|2010          |25923|
|2011          |25923|
|2012          |25923|
|2013          |25923|
|2014          |25923|
|2015          |25923|
|2016          |25923|
|2017          |25923|
|2018          |25923|
|2019          |25923|
+--------------+-----+



<b>Camada Gold - gold/educacao_superior/indicadores_fluxo_educacao_superior</b>

In [3]:
df_educacao = spark.read.format('parquet') \
   .load('s3a://gold/educacao_superior/indicadores_fluxo_educacao_superior')
df_educacao.show(5, False)

+------------------+-----------------------------------+-------------------------------+-----------------------------+----------------------------+--------------------------+----------------------+--------------------+------------------------------+----------------------------+-------------------------------+-----------------------------+----------------------+--------------------+--------------------------+------------------------+-----------------------------+---------------------------+-------------------------------------------+-----------------------------------------+--------------------------------------------------+------------------------------------------------+------------+--------------+-------------------------+------------------------+-------------------------------+-------------------------------+----------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------+-----------------